In [15]:
# #@test {"skip": true}
# !pip install --quiet --upgrade tensorflow_federated_nightly
# !pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [16]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle

tff.backends.reference.set_reference_context()

In [17]:
from tensorflow.keras import layers
from tensorflow.keras import activations

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

import seaborn as sns
sns.set_style("whitegrid")

In [18]:
origTra = open('./Datasets/pendigits/pendigits-orig.tra', 'r').read()
tra     = open('./Datasets/pendigits/pendigits.tra', 'r').read()
origTes = open('./Datasets/pendigits/pendigits-orig.tes', 'r').read()
tes     = open('./Datasets/pendigits/pendigits.tes', 'r').read()

In [19]:
listOrigTra = []
for i in origTra.split('COMMENT ')[1:]:
    listOrigTra.append(i.split()[1])

In [20]:
listTra = []
for i in tra.split('\n'):
    listTra.append(i[-1])

In [21]:
listOrigTes = []
for i in origTes.split('COMMENT ')[1:]:
    listOrigTes.append(i.split()[1])

In [22]:
listTes = []
for i in tes.split('\n'):
    listTes.append(i[-1])

In [23]:
columns = ['Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6', 'Input7', 'Input8', 'Input9', 'Input10', 'Input11', 'Input12', 'Input13', 'Input14', 'Input15', 'Input16', 'digit']

In [24]:
df_tra = pd.read_csv('./Datasets/pendigits/pendigits.tra', delimiter = ',', names=columns , header=None)
df_tra.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [25]:
df_tra['writer'] = listOrigTra
df_tra.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [26]:
df_tes = pd.read_csv('./Datasets/pendigits/pendigits.tes', delimiter = ',', names=columns , header=None)
df_tes.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9


In [4]:
# mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
# [(x.dtype, x.shape) for x in mnist_train]

[(dtype('uint8'), (60000, 28, 28)), (dtype('uint8'), (60000,))]

In [6]:
NUM_EXAMPLES_PER_USER = 250
BATCH_SIZE = 25
NUM_OF_CLIENTS = 30
# NUM_OF_CLIENTS_ARRAY = [0,0,0,0, 1,1,1,1, 2,2,2,2, 3,3,3,3, 4,4,4,4, 5,5,5,5, 6,6,6,6, 7,7,7,7, 8,8,8,8, 9,9,9,9]
# NUM_OF_CLIENTS_ARRAY = [0,1,2,3,4,5,6,7,8,9]

In [7]:
# source_train_X, source_train_Y = shuffle(mnist_train[0], mnist_train[1], random_state=0)
# source_test_X, source_test_Y = shuffle(mnist_test[0], mnist_test[1], random_state=0)

In [8]:
def get_data_for_digit_iid(source_X, source_Y):
    output_sequence = []
    for i in range(0, min(NUM_EXAMPLES_PER_USER, len(source_X)), BATCH_SIZE):
        batch_samples_X = source_X[i:i + BATCH_SIZE]
        batch_samples_Y = source_Y[i:i + BATCH_SIZE]
        output_sequence.append({
            'x': np.array([xx.flatten() / 255.0 for xx in batch_samples_X],dtype=np.float32),
            'y': np.array([yy for yy in batch_samples_Y], dtype=np.int32)
        })
    return output_sequence

xtr = int(len(source_train_X)/NUM_OF_CLIENTS)
ytr = int(len(source_train_Y)/NUM_OF_CLIENTS)

xte = int(len(source_test_X)/NUM_OF_CLIENTS)
yte = int(len(source_test_Y)/NUM_OF_CLIENTS)

# federated_train_data = [get_data_for_digit_iid(source_train_X[(d*xtr):(d*xtr+xtr)], source_train_Y[(d*ytr):(d*ytr+ytr)]) for d in range(NUM_OF_CLIENTS)]

# federated_test_data = [get_data_for_digit_iid(source_test_X[(d*xte):(d*xte+xte)], source_test_Y[(d*yte):(d*yte+yte)]) for d in range(NUM_OF_CLIENTS)]

In [9]:
def get_data_for_digit_niid(source, digit):
    output_sequence = []
    all_samples = [i for i, d in enumerate(source[1]) if d == digit]
    for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
        batch_samples = all_samples[i:i + BATCH_SIZE]
        output_sequence.append({
            'x': np.array([source[0][i].flatten() / 255.0 for i in batch_samples],dtype=np.float32),
            'y': np.array([source[1][i] for i in batch_samples], dtype=np.int32)
        })
    return output_sequence


# federated_train_data = [get_data_for_digit_niid(mnist_train, d) for d in NUM_OF_CLIENTS_ARRAY]

# federated_test_data = [get_data_for_digit_iid(source_test_X[(d*xte):(d*xte+xte)], source_test_Y[(d*yte):(d*yte+yte)]) for d in range(NUM_OF_CLIENTS)]

In [10]:
# mnist_train[0].shape

In [11]:
# federated_train_data

In [12]:
# federated_train_data[0][0].get('y').shape

In [13]:
# source_train_X.shape

In [14]:
# federated_train_data2[0][0].get('x').shape

In [15]:
# federated_train_data2[0][0].get('y').shape

In [27]:
BATCH_SPEC = collections.OrderedDict(
    x=tf.TensorSpec(shape=[None, 16], dtype=tf.float32),
    y=tf.TensorSpec(shape=[None], dtype=tf.int32))
BATCH_TYPE = tff.to_type(BATCH_SPEC)

str(BATCH_TYPE)

'<x=float32[?,16],y=int32[?]>'

In [28]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

str(LOCAL_DATA_TYPE)

'<x=float32[?,16],y=int32[?]>*'

In [29]:
MODEL_SPEC = collections.OrderedDict(
    weights=tf.TensorSpec(shape=[10, 10], dtype=tf.float32),
    bias=tf.TensorSpec(shape=[10], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

print(MODEL_TYPE)

<weights=float32[10,10],bias=float32[10]>


In [30]:
SERVER_MODEL_TYPE = tff.FederatedType(MODEL_TYPE, tff.SERVER)
CLIENT_DATA_TYPE = tff.FederatedType(LOCAL_DATA_TYPE, tff.CLIENTS)

Client

In [20]:
@tf.function
def forward_pass(model, batch):
    predicted_y = tf.nn.softmax(tf.matmul(batch['x'], model['weights']) + model['bias'])
    return -tf.reduce_mean(
        tf.reduce_sum(
            tf.one_hot(batch['y'], 10) * tf.math.log(predicted_y), axis=[1]))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
    return forward_pass(model, batch)

In [21]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
    model_vars = collections.OrderedDict([
        (name, tf.Variable(name=name, initial_value=value))
        for name, value in initial_model.items()
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    
    @tf.function
    def _train_on_batch(model_vars, batch):
        # Perform one step of gradient descent using loss from `batch_loss`.
        with tf.GradientTape() as tape:
            loss = forward_pass(model_vars, batch)
        grads = tape.gradient(loss, model_vars)
        optimizer.apply_gradients(zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
        return model_vars
  
    return _train_on_batch(model_vars, batch)

In [22]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):
    
    # Mapping function to apply to each batch.
    @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
    def batch_fn(model, batch):
        return batch_train(model, batch, learning_rate)
    return tff.sequence_reduce(all_batches, initial_model, batch_fn)

In [23]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval(model, all_batches):
    return tff.sequence_sum(
        tff.sequence_map(
            tff.federated_computation(lambda b: batch_loss(model, b), BATCH_TYPE),all_batches))

Server

In [24]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model, data):
    return tff.federated_mean(tff.federated_map(local_eval, [tff.federated_broadcast(model), data]))

In [25]:
SERVER_FLOAT_TYPE = tff.FederatedType(tf.float32, tff.SERVER)


@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE,
                           CLIENT_DATA_TYPE)
def federated_train(model, learning_rate, data):
#     client_output = tff.federated_map(local_train, [
#             tff.federated_broadcast(model),
#              tff.federated_broadcast(learning_rate), data
#         ])
    return tff.federated_mean(tff.federated_map(local_train, [
            tff.federated_broadcast(model),
             tff.federated_broadcast(learning_rate), data
        ]))

In [26]:
initial_model = collections.OrderedDict(
    weights=np.zeros([10, 10], dtype=np.float32),
    bias=np.zeros([10], dtype=np.float32))

In [27]:
def modelR(weights, biases, data):
  count = 0
  avg = 0
  for j in range(len(federated_test_data)):
    l = [np.where(i==max(i))[0] for i in activations.sigmoid(np.matmul(federated_test_data[j].get('x'), model.get('weights'))).numpy()]
    Y = federated_test_data[0][j].get('y')
    
    for i in range(len(Y)):
      avg+=1
      if l[i] == Y[i]:
        count+= 1
  return count/avg

In [28]:
# model = initial_model
# learning_rate = 0.1
# for round_num in range(5):
#     model = federated_train(model, learning_rate, federated_train_data)
#     learning_rate = learning_rate * 0.9
#     loss = federated_eval(model, federated_train_data)
#     print('round {}, loss={}, accuracy={}'.format(round_num, loss, modelR(model.get('weights'), model.get('bias'), federated_test_data)))

In [ ]:
accuracy = 0
model = initial_model
learning_rate = 0.1
count = 0

accuracy_list = []
for i in range(10):
    model = federated_train(model, learning_rate, federated_train_data)
    learning_rate = learning_rate * 0.9
    loss = federated_eval(model, federated_train_data)
    accuracy = modelR(model.get('weights'), model.get('bias'), federated_test_data)
    count+=1
    accuracy_list.append(accuracy)
    print('round {}, loss={}, accuracy={}'.format(count, loss, accuracy)) 

# pickle.dump(accuracy_list, open('90_clients', 'wb'))

round 1, loss=12.019745826721191, accuracy=0.7837837837837838
round 2, loss=9.80299186706543, accuracy=0.8108108108108109
round 3, loss=8.504426002502441, accuracy=0.8468468468468469
round 4, loss=7.675743103027344, accuracy=0.8558558558558559
round 5, loss=7.108245372772217, accuracy=0.8558558558558559


In [ ]:
# plt.figure(figsize=(8,6))
# plt.plot(pickle.load(open('20_clients', 'rb')), label="20")
# plt.plot(pickle.load(open('30_clients', 'rb')), label="30")
# plt.plot(pickle.load(open('50_clients', 'rb')), label="50")
# plt.plot(pickle.load(open('90_clients', 'rb')), label="90")


# plt.legend(loc="upper left")
# plt.show()

In [ ]:
# def modelR(weights, biases, data):
#   count = 0
#   avg = 0
#   for j in range(len(federated_test_data[0])):
#     l = [np.where(i==max(i))[0][0] for i in activations.sigmoid(np.matmul(federated_test_data[0][j].get('x'), model.get('weights'))).numpy()]
#     Y = federated_test_data[0][j].get('y')
#     print('l={}, Y={}'.format(l, Y))
    
#     for i in range(len(Y)):
#       avg+=1
#       if l[i] == Y[i]:
#         count+= 1
#   return count/avg


In [ ]:
# modelR(model.get('weights'), model.get('bias'), federated_test_data)

#